In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
spark = SparkSession \
    .builder \
    .appName("Exploring Data") \
    .getOrCreate()

In [11]:
path = "/Users/km4657/code/python/sf-crime-analysis/police-department-calls-for-service.json"

event_log = spark.read.json(path, multiLine=True)




In [12]:
event_log.printSchema()

root
 |-- address: string (nullable = true)
 |-- address_type: string (nullable = true)
 |-- agency_id: string (nullable = true)
 |-- call_date: string (nullable = true)
 |-- call_date_time: string (nullable = true)
 |-- call_time: string (nullable = true)
 |-- city: string (nullable = true)
 |-- common_location: string (nullable = true)
 |-- crime_id: string (nullable = true)
 |-- disposition: string (nullable = true)
 |-- offense_date: string (nullable = true)
 |-- original_crime_type_name: string (nullable = true)
 |-- report_date: string (nullable = true)
 |-- state: string (nullable = true)



In [13]:
event_log.take(5)

[Row(address='Geary Bl/divisadero St', address_type='Intersection', agency_id='1', call_date='2018-12-31T00:00:00.000', call_date_time='2018-12-31T23:57:00.000', call_time='23:57', city='San Francisco', common_location='', crime_id='183653763', disposition='ADM', offense_date='2018-12-31T00:00:00.000', original_crime_type_name='Traffic Stop', report_date='2018-12-31T00:00:00.000', state='CA'),
 Row(address='100 Blk Howard St', address_type='Geo-Override', agency_id='1', call_date='2018-12-31T00:00:00.000', call_date_time='2018-12-31T23:54:00.000', call_time='23:54', city='San Francisco', common_location='', crime_id='183653756', disposition='CIT', offense_date='2018-12-31T00:00:00.000', original_crime_type_name='Traf Violation Cite', report_date='2018-12-31T00:00:00.000', state='CA'),
 Row(address='3300 Block Of 20th Av', address_type='Common Location', agency_id='1', call_date='2018-12-31T00:00:00.000', call_date_time='2018-12-31T23:49:00.000', call_time='23:49', city='San Francisco',

In [14]:
event_log.describe().show()


+-------+------------------+---------------+---------+--------------------+--------------------+---------+-----------+------------------+--------------------+-----------+--------------------+------------------------+--------------------+------+
|summary|           address|   address_type|agency_id|           call_date|      call_date_time|call_time|       city|   common_location|            crime_id|disposition|        offense_date|original_crime_type_name|         report_date| state|
+-------+------------------+---------------+---------+--------------------+--------------------+---------+-----------+------------------+--------------------+-----------+--------------------+------------------------+--------------------+------+
|  count|            199999|         199999|   199999|              199999|              199999|   199999|     199999|            199999|              199999|     199999|              199999|                  199999|              199999|199999|
|   mean| 133.090909

In [16]:
event_log.describe("original_crime_type_name").show()

+-------+------------------------+
|summary|original_crime_type_name|
+-------+------------------------+
|  count|                  199999|
|   mean|      4069.9766435812558|
| stddev|       9006.415090272823|
|    min|           "Bumper Jack"|
|    max|                      `X|
+-------+------------------------+



In [42]:
event_log.select("original_crime_type_name").dropDuplicates().show()


+------------------------+
|original_crime_type_name|
+------------------------+
|                 Footage|
|                   Sword|
|                 415/909|
|            Physical/601|
|                   915/x|
|              Ne Tx 0123|
|                 488/594|
|                   Drink|
|                 Dealing|
|                 Erratic|
|          594 W/ Bb Guns|
|            Passing Call|
|                     853|
|          Person W/knife|
|                     800|
|                      Gz|
|               Loud Bass|
|                   2200e|
|            Dropped Call|
|              Music/ 909|
+------------------------+
only showing top 20 rows



In [36]:
#event_log.select("original_crime_type_name").dropDuplicates().sort("original_crime_type_name").show(500)



events_by_type = event_log.groupby(event_log.original_crime_type_name).count().orderBy('count', ascending=False).show(100)

+------------------------+-----+
|original_crime_type_name|count|
+------------------------+-----+
|            Passing Call|35180|
|            Traffic Stop|13538|
|     Traf Violation Cite|10921|
|       Suspicious Person|10007|
|      Homeless Complaint| 6072|
|              Trespasser| 5840|
|           Audible Alarm| 5775|
|                  22500e| 5651|
|        Well Being Check| 5552|
|         Muni Inspection| 5509|
|      Suspicious Vehicle| 5031|
|         Fight No Weapon| 4199|
|          Noise Nuisance| 4072|
|      Auto Boost / Strip| 2969|
|      Traf Violation Tow| 2915|
|                    Poss| 2851|
|      Mentally Disturbed| 2585|
|             Petty Theft| 2549|
|          Meet W/citizen| 2414|
|       Assault / Battery| 2356|
|                   Drugs| 2003|
|                     601| 1837|
|          Complaint Unkn| 1676|
|                    500e| 1519|
|                Burglary| 1423|
|          Stolen Vehicle| 1408|
|    Threats / Harassment| 1380|
|         

In [37]:
events_by_location = event_log.groupby(event_log.address_type).count().orderBy('count', ascending=False).show(100)

+---------------+------+
|   address_type| count|
+---------------+------+
|Premise Address|102791|
|   Intersection| 64091|
|Common Location| 24446|
|   Geo-Override|  8671|
+---------------+------+



In [43]:
distinct_table = event_log.dropDuplicates(["original_crime_type_name", "disposition"]).show()

+--------------------+---------------+---------+--------------------+--------------------+---------+-------------+--------------------+---------+-----------+--------------------+------------------------+--------------------+-----+
|             address|   address_type|agency_id|           call_date|      call_date_time|call_time|         city|     common_location| crime_id|disposition|        offense_date|original_crime_type_name|         report_date|state|
+--------------------+---------------+---------+--------------------+--------------------+---------+-------------+--------------------+---------+-----------+--------------------+------------------------+--------------------+-----+
| 100 Block Of 6th St|Common Location|        1|2018-10-30T00:00:...|2018-10-30T08:19:...|    08:19|San Francisco| Henry Hotel Sro, Sf|183030788|        GOA|2018-10-30T00:00:...|                261 Poss|2018-10-30T00:00:...|   CA|
|100 Block Of New ...|Premise Address|        1|2018-11-02T00:00:...|2018-11

In [44]:
 distinct_table = event_log.select("original_crime_type_name", "disposition")

In [47]:
distinct_table.show()

+------------------------+------------+
|original_crime_type_name| disposition|
+------------------------+------------+
|            Traffic Stop|         ADM|
|     Traf Violation Cite|         CIT|
|            Passing Call|         HAN|
|           Audible Alarm|         PAS|
|            Traffic Stop|         CIT|
|            Passing Call|          ND|
|            Traffic Stop|         CIT|
|            Traffic Stop|         HAN|
|            Passing Call|Not recorded|
|         Fight No Weapon|         GOA|
|            Traffic Stop|         HAN|
|            Passing Call|         HAN|
|            Passing Call|         HAN|
|            Traffic Stop|         CIT|
|                     Att|         REP|
|      Auto Boost / Strip|         REP|
|            Passing Call|         HAN|
|            Traffic Stop|         ADV|
|            Passing Call|         HAN|
|      Suspicious Vehicle|         ADV|
+------------------------+------------+
only showing top 20 rows



In [48]:
agg_df = distinct_table.groupby(event_log.original_crime_type_name).count().orderBy('count', ascending=False).show(100)

+------------------------+-----+
|original_crime_type_name|count|
+------------------------+-----+
|            Passing Call|35180|
|            Traffic Stop|13538|
|     Traf Violation Cite|10921|
|       Suspicious Person|10007|
|      Homeless Complaint| 6072|
|              Trespasser| 5840|
|           Audible Alarm| 5775|
|                  22500e| 5651|
|        Well Being Check| 5552|
|         Muni Inspection| 5509|
|      Suspicious Vehicle| 5031|
|         Fight No Weapon| 4199|
|          Noise Nuisance| 4072|
|      Auto Boost / Strip| 2969|
|      Traf Violation Tow| 2915|
|                    Poss| 2851|
|      Mentally Disturbed| 2585|
|             Petty Theft| 2549|
|          Meet W/citizen| 2414|
|       Assault / Battery| 2356|
|                   Drugs| 2003|
|                     601| 1837|
|          Complaint Unkn| 1676|
|                    500e| 1519|
|                Burglary| 1423|
|          Stolen Vehicle| 1408|
|    Threats / Harassment| 1380|
|         

In [52]:
event_log.select("disposition").dropDuplicates().show(100)

+------------+
| disposition|
+------------+
|         CHP|
|         DUP|
|         ABA|
|         INC|
|         GEN|
|         TH2|
|          22|
|         NOM|
|         HOT|
|         GOA|
|         CIT|
|         HAN|
|         ADM|
|         ENC|
|         FIR|
|          ND|
|         SFD|
|         PAS|
|         UTL|
|         NCR|
|         CAN|
|         ARR|
|         OTS|
|         TOW|
|         ADV|
|         REP|
|         CRM|
|Not recorded|
+------------+



In [53]:
path = "/Users/km4657/code/python/sf-crime-analysis/radio_code.json"

code_log = spark.read.json(path, multiLine=True)

In [54]:
code_log.select("disposition_code").dropDuplicates().show(100)

+----------------+
|disposition_code|
+----------------+
|             ABA|
|              22|
|             NOM|
|             GOA|
|             CIT|
|             HAN|
|             ADM|
|              ND|
|             SFD|
|             PAS|
|             UTL|
|             NCR|
|             CAN|
|             ARR|
|             VAS|
|             ADV|
|             REP|
|             CRM|
|             CSA|
+----------------+

